[Reference](https://medium.datadriveninvestor.com/mastering-streamlit-the-biggest-flaw-8387660498b7)

In [1]:
import streamlit as st

def main(test):
    st.set_page_config(layout="wide")
    state = _get_state()
    
    st.sidebar.title(":floppy_disk: Dashboard")
    page = st.sidebar.radio("Select your page", tuple(pages.keys()))
    
    #Removing and add pages 
    pages = {
        "Login": page_login,
    }

    #For skipping sign-ups for testing 
    if test == "testing_dashboard":
        state.login = True
        state.user_name = getpass.getuser()

    if state.login:
        pages.pop("Login")
        pages["Crypto"] = page_crypto
        pages["Dashboard"] = page_dashboard
        pages["Graphs"] = page_graph
        # pages["Settings"] = page_settings

    # Display the selected page with the session state
    pages[page](state)

    # Mandatory to avoid rollbacks with widgets, must be called at the end of your app
    state.sync()

In [2]:
from streamlit.hashing import _CodeHasher

try:
    # Before Streamlit 0.65
    from streamlit.ReportThread import get_report_ctx
    from streamlit.server.Server import Server
except ModuleNotFoundError:
    # After Streamlit 0.65
    from streamlit.report_thread import get_report_ctx
    from streamlit.server.server import Server


def _get_state(hash_funcs=None):
    session = _get_session()

    if not hasattr(session, "_custom_session_state"):
        session._custom_session_state = _SessionState(session, hash_funcs)

    return session._custom_session_state

class _SessionState:

    def __init__(self, session, hash_funcs):
        """Initialize SessionState instance."""
        self.__dict__["_state"] = {
            "data": {},
            "hash": None,
            "hasher": _CodeHasher(hash_funcs),
            "is_rerun": False,
            "session": session,
        }

    def __call__(self, **kwargs):
        """Initialize state data once."""
        for item, value in kwargs.items():
            if item not in self._state["data"]:
                self._state["data"][item] = value

    def __getitem__(self, item):
        """Return a saved state value, None if item is undefined."""
        return self._state["data"].get(item, None)
        
    def __getattr__(self, item):
        """Return a saved state value, None if item is undefined."""
        return self._state["data"].get(item, None)

    def __setitem__(self, item, value):
        """Set state value."""
        self._state["data"][item] = value

    def __setattr__(self, item, value):
        """Set state value."""
        self._state["data"][item] = value
    
    def clear(self):
        """Clear session state and request a rerun."""
        self._state["data"].clear()
        self._state["session"].request_rerun()
    
    def sync(self):
        """Rerun the app with all state values up to date from the beginning to fix rollbacks."""

        # Ensure to rerun only once to avoid infinite loops
        # caused by a constantly changing state value at each run.
        #
        # Example: state.value += 1
        if self._state["is_rerun"]:
            self._state["is_rerun"] = False
        
        elif self._state["hash"] is not None:
            if self._state["hash"] != self._state["hasher"].to_bytes(self._state["data"], None):
                self._state["is_rerun"] = True
                self._state["session"].request_rerun()

        self._state["hash"] = self._state["hasher"].to_bytes(self._state["data"], None)

In [3]:
import pandas as pd
df = pd.DataFrame(
    np.random.randn(50, 20),
    columns=('col %d' % i for i in range(20))
)
st.dataframe(df)

In [4]:
@st.cache
def load_data(nrows):
    data = pd.read_csv(DATA_URL, nrows=nrows)
    return data

In [5]:
stock_table = st.empty()#Load dataframe from local database 
dataframe = create_df(get_stocks_df(DATABASE_FILE_LOCATION))#round values to 2dp
dataframe = dataframe.round(2)#Make negative values red in colour
stock_table.table(dataframe.style.applymap(color_negative_red))

In [6]:
st.header("Stock Purchase")stock_name = st.empty()
stock_quantity = st.empty()
stock_bought_price = st.empty()
stock_fees = st.empty()stock_name = str(stock_name.text_input("Stock Code E.g AAPL, BABA").strip().upper())stock_quantity = stock_quantity.text_input("Quantity").strip()stock_bought_price = stock_bought_price.text_input("Bought Price (Stock Market's Currency: e.g STI in SGD, BABA in USD)").strip()stock_fees = stock_fees.text_input("Extra Fees (Commissions etc)").strip()add_button = st.empty()add_button_state = add_button.button("Submit")

In [7]:
import sqlite3
from sqlite3 import Error


DATABASE_FILE_LOCATION = os.getcwd()+"\pythonsqlite.db" ## in C:\User\User\Desktop

TABLE_DIC = {'stocks':'stocks','crypto':'crypto','stock_trans':'stock_transaction','crypto_trans':'crypto_transaction'} ## Stocks always first, cryto always second

def create_connection(db_file):
    """ create a database connection to a SQLite database """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)

        if (conn.execute("SELECT name FROM sqlite_master").fetchall() not in TABLE_DIC.values()):
            print("Creating "+ str(TABLE_DIC.values()) +" Table")

        conn.execute("""CREATE TABLE IF NOT EXISTS """ + TABLE_DIC['stock_trans'] + """ (    
                    Stock                       TEXT     PRIMARY KEY     NOT NULL,
                    Bought_Price                REAL                     NOT NULL,
                    Currency                    TEXT                     NOT NULL,
                    Fees                        REAL                     NOT NULL,
                    Quantity                    REAL                     NOT NULL);""")
    
        conn.execute("""CREATE TABLE IF NOT EXISTS """ + TABLE_DIC['stocks'] + """ (
                    Stock                       TEXT     PRIMARY KEY     NOT NULL,
                    Bought_Price_Avg            REAL                     NOT NULL,
                    Currency                    TEXT                     NOT NULL,
                    Fees                        REAL                     NOT NULL,
                    Quantity                    REAL                     NOT NULL);""")

        conn.execute("""CREATE TABLE IF NOT EXISTS """ + TABLE_DIC['crypto_trans'] + """ (
                    Coin_Pair                   TEXT     PRIMARY KEY     NOT NULL,
                    Bought_Price                REAL                     NOT NULL,
                    Currency                    TEXT                     NOT NULL,
                    Fees                        REAL                     NOT NULL,
                    Quantity                    REAL                     NOT NULL);""")

        conn.execute("""CREATE TABLE IF NOT EXISTS """ + TABLE_DIC['crypto'] + """ (
                    Coin_Pair                   TEXT     PRIMARY KEY     NOT NULL,
                    Bought_Price_Avg            REAL                     NOT NULL,
                    Currency                    TEXT                     NOT NULL,
                    Fees                        REAL                     NOT NULL,
                    Quantity                    REAL                     NOT NULL);""")

        print("Successfully created "+ str(TABLE_DIC.values()) +" Table")

    except Error as e:
        print(e)
    finally:
        if conn:
            conn.close()

In [8]:
import sqlite3
from sqlite3 import Error

def add_items_to_database(db_file,stock_name,stock_quantity,stock_bought_price_weighted_avg,stock_fees,stock_currency):
    try:
        conn = sqlite3.connect(db_file)
        print("INSERT INTO stocks (Stock,Bought_Price_Avg,Currency,Fees,Quantity) VALUES ('"+str(stock_name)+"',"+str(stock_bought_price_weighted_avg)+",'"+str(stock_currency)+"',"+str(stock_fees)+","+str(stock_quantity)+")")
        conn.execute("INSERT INTO stocks (Stock,Bought_Price_Avg,Currency,Fees,Quantity) VALUES ('"+stock_name+"',"+stock_bought_price_weighted_avg+",'"+stock_currency+"',"+stock_fees+","+stock_quantity+")")
        conn.commit()
    except Error as e:
        print(e)
    finally:
        if conn:
            conn.close()


def check_existence_of_stock_name(db_file,stock_name):
    try:
        conn = sqlite3.connect(db_file)
        return pd.read_sql_query(("SELECT * FROM stocks WHERE Stock = "+stock_name),conn)
    except Error as e:
        raise Exception('stock does not exist yet')
        print(e)
    finally:
        if conn:
            conn.close()

In [9]:
# Summary which deals with different markets
    st.write('Summary')
    summary_df = df.reindex(columns=['Profit/Loss','Profit/Loss (%)','Market Value','Total Spent','Currency']).groupby(['Currency']).agg('sum')
    summary_df['Profit/Loss (%)'] = ((summary_df['Market Value'] - summary_df['Total Spent']) / summary_df['Market Value']) *100
    st.table(summary_df)

    # Final Tally (SGD)
    st.write('Final Table (SGD)')
    final_df = df.reindex(columns=['Market Value (SGD)','Total Spent (SGD)','Currency']).groupby(['Currency']).agg('sum')
    final_df_sgd = final_df.sum()
    final_df_sgd['Profit/Loss (SGD)'] = (final_df_sgd['Market Value (SGD)'] - final_df_sgd['Total Spent (SGD)'])
    final_df_sgd['Profit/Loss (%) (SGD)'] = round(((final_df_sgd['Market Value (SGD)'] - final_df_sgd['Total Spent (SGD)']) / final_df_sgd['Market Value (SGD)']) * 100,2)
    st.table(final_df_sgd)

    # Sort df in terms of currency then profit/loss amounts 
    df.sort_values(by=['Currency','Profit/Loss'],ascending=False,inplace=True)
    df.reset_index(inplace=True,drop=True) 

    #Plot Asset Allocation
    fig = px.pie(df, values=df['Total Spent (SGD)'], names=df['Stock'], title='Asset Allocation (SGD)')
    st.plotly_chart(fig)
    for index,row in final_df.iterrows():
        st.write(index+" is "+str(round((row['Total Spent (SGD)']/final_df_sgd['Total Spent (SGD)'])*100,2))+" %")

    #Plot Profit Allocation
    fig = px.pie(df, values=df['Market Value (SGD)']-df['Total Spent (SGD)'], names=df['Stock'], title='Profit Allocation (SGD)')
    st.plotly_chart(fig)
    for index,row in final_df.iterrows():
        st.write(index+" is "+str(round((row['Market Value (SGD)']/final_df_sgd['Market Value (SGD)'])*100,2))+" %")

    #Plot Stock profit comparison 
    fig = px.bar(df, y=df['Market Value (SGD)']-df['Total Spent (SGD)'], x=df['Stock'], title='Stock profit comparison (SGD)')
    fig.update_layout(xaxis={'categoryorder':'total descending'})
    st.plotly_chart(fig)